Everything to Get Current Season

Scrape

In [7]:
import pandas as pd
import csv
import os
from pathlib import Path

# Set working directory to project root if run from notebooks/
if Path.cwd().name == "notebooks":
    os.chdir("..")

# Define folders – assume they already exist
raw_data_dir = Path("data/Raw")
cleaned_data_dir = Path("data/Cleaned")
metrics_data_dir = Path("data/Metrics")
current_season_dir = metrics_data_dir / "CurrentSeasonTeams"

url = 'https://www.basketball-reference.com/wnba/years/2025_totals.html'
df2025 = pd.read_html(url)[0]  # First table is the player totals
df2025.to_csv(raw_data_dir / '2025Data.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')
df2025 = df2025[df2025['Player'] != 'Player']  # Drop header rows repeated in table
df2025 = df2025.drop_duplicates()        # Remove duplicate entries, if any
tot_players = df2025[df2025['Team'] == 'TOT']['Player']
df2025 = pd.concat([
    df2025[df2025['Team'] == 'TOT'],                              # all TOT rows
    df2025[~df2025['Player'].isin(tot_players) & (df2025['Team'] != 'TOT')]  # single-team players
], ignore_index=True)
df2025['MP'] = pd.to_numeric(df2025['MP'])
df2025 = df2025.drop('G.1', axis=1)
df2025 = df2025.drop('MP.1', axis=1)

cols_to_exclude = ['Player', 'Team', 'Pos']
cols_to_convert = df2025.columns.difference(cols_to_exclude)

df2025[cols_to_convert] = df2025[cols_to_convert].apply(pd.to_numeric, errors='coerce')

df2025

,Player,Team,Pos,G,MP,GS,FG,FGA,FG%,3P,...,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Grace Berger,TOT,G,6,105,0,6,26,0.231,0,...,2,0.000,4,19,13,4,1,6,14,12
1,Chloe Bibby,TOT,F,12,135,0,17,48,0.354,14,...,9,0.889,4,22,7,7,1,4,12,56
2,DeWanna Bonner,TOT,G-F,20,457,4,60,148,0.405,23,...,56,0.857,17,86,25,15,4,15,23,191
3,DiJonai Carrington,TOT,G-F,22,544,13,83,223,0.372,16,...,64,0.797,42,108,38,25,11,53,57,233
4,Kaila Charles,TOT,G-F,21,344,4,41,99,0.414,15,...,29,0.828,35,87,22,12,5,20,39,121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Gabby Williams,SEA,F,31,1019,31,152,347,0.438,40,...,58,0.879,19,145,140,79,13,64,46,395
170,Kiana Williams,PHO,G,6,69,0,12,24,0.500,5,...,2,0.500,0,7,7,1,0,1,2,30
171,A'ja Wilson,LVA,C,27,817,27,208,426,0.488,11,...,194,0.835,51,248,89,44,59,63,58,589
172,Jackie Young,LVA,G,31,924,31,179,395,0.453,51,...,126,0.913,37,128,141,41,9,73,82,524


Adding DRB

In [8]:
df2025['DRB'] = df2025['TRB'] - df2025['ORB']

# Define target and reference columns
col_to_move = 'DRB'
before_col = 'TRB'

# Get current column order and remove the column to move
cols = df2025.columns.tolist()
cols.remove(col_to_move)

# Find index to insert the column before the target
insert_at = cols.index(before_col)

# Insert the column at the desired position
cols.insert(insert_at, col_to_move)

# Reorder the DataFrame
df2025 = df2025[cols]

df2025.to_csv(cleaned_data_dir / '2025Data.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

Metrics

In [9]:
df2025 = pd.read_csv(cleaned_data_dir / '2025Data.csv')

df2025['Efficiency'] = (1/df2025['G']) * (df2025['PTS'] + df2025['TRB'] + df2025['AST'] + df2025['STL'] - df2025['TOV'] - (df2025['FGA'] - df2025['FG']) - (df2025['FTA'] - df2025['FT']))

df2025['WinScore'] = (1/df2025['G']) * (df2025['PTS'] + df2025['TRB'] + df2025['STL'] + (df2025['AST'] * 0.5) + (df2025['BLK'] * 0.5) - df2025['FGA'] - df2025['TOV'] - (df2025['FTA'] * 0.5) - (df2025['PF'] * 0.5))

df2025['NBAPER'] = .68 + ((1/df2025['MP']) * ((26.77 * df2025['FG']) - (35.58 * df2025['FGA']) + (26.32 * df2025['FT']) - (25.47 * df2025['FTA']) + (42.74 * df2025['ORB']) + (11.45 * df2025['DRB']) + (29.7 * df2025['AST']) + (47.3 * df2025['STL']) + (38 * df2025['BLK']) - (47.9 * df2025['TOV']) - (18.8 * df2025['PF']) + (41.1 * df2025['PTS'])))

df2025['WNBAPER'] = .3289 + ((1/df2025['MP']) * ((53.35 * df2025['FG']) - (29.28 * df2025['FGA']) + (15.27 * df2025['FT']) - (25.34 * df2025['FTA']) + (47.72 * df2025['ORB']) + (10.95 * df2025['DRB']) + (31.61 * df2025['AST']) + (50.72 * df2025['STL']) + (50.68 * df2025['BLK']) - (45.88 * df2025['TOV']) - (61.23 * df2025['PF']) + (41.1 * df2025['PTS'])))

print("Columns before filtering (2025):", df2025.columns.tolist())

expected_cols = ['Player', 'Team', 'Pos', 'WinScore', 'Efficiency', 'NBAPER', 'WNBAPER']

df2025 = df2025[[col for col in expected_cols if col in df2025.columns]]

df2025.to_csv(current_season_dir / '2025Metrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

Columns before filtering (2025): ['Player', 'Team', 'Pos', 'G', 'MP', 'GS', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Efficiency', 'WinScore', 'NBAPER', 'WNBAPER']


Teamless

In [10]:
df2025 = pd.read_csv(current_season_dir / '2025Metrics.csv')

df2025teamless = df2025.drop(columns=['Team'])

df2025teamless = df2025teamless.round(3)

df2025teamless.to_csv(metrics_data_dir / '2025Metrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

Team Specific

In [11]:
df2025 = pd.read_csv(current_season_dir / '2025Metrics.csv')

df2025 = df2025.round(3)

# The column to split on
split_column = "Team"

# Directory to save files (ensure it exists or create it)
output_dir = current_season_dir

# Split and save
for group, group_df in df2025.groupby(split_column):
    filename = f"{output_dir}/{group}.csv"
    group_df = group_df.drop(columns=['Team'])  # Drop 'Team' column
    group_df.to_csv(filename, index=False, sep=',', encoding='utf-8', lineterminator='\n')

Combined DFs

In [12]:

df2023 = pd.read_csv(metrics_data_dir / '2023Metrics.csv')

df2024 = pd.read_csv(metrics_data_dir / '2024Metrics.csv')

df2025 = pd.read_csv(metrics_data_dir / '2025Metrics.csv')

combined = pd.concat([
    df2023.assign(Season=2023),
    df2024.assign(Season=2024),
    df2025.assign(Season=2025)
])

combined[['Player', 'Season', 'Pos', 'WinScore', 'Efficiency', 'NBAPER', 'WNBAPER']].to_csv(metrics_data_dir / 'AllPlayersMetrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')